# MODELING NOTEBOOK

library import

In [2]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
from tensorflow import keras
import tensorflow as tf
import warnings
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import cross_val_score

from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectPercentile
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
plt.rc('font', family=fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()) # for Windows OS user
import datetime
import os

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge
from tensorflow import keras
from scipy.stats.mstats import gmean
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")

### read features

In [4]:
y = pd.read_csv(os.path.abspath("../input")+'/y_train.csv', encoding='cp949')

In [5]:
tst = pd.read_csv(os.path.abspath("../input")+'/X_test.csv', encoding='cp949')
IDtest = tst.custid.unique()

In [ ]:
'''
# 우리가 직접 진행한 feature engineering
X_train_select = pd.read_csv(os.path.abspath("../features")+'/X_train_select_hj.csv', encoding='cp949')
X_test_select = pd.read_csv(os.path.abspath("../features")+'/X_test_select_hj.csv', encoding='cp949')
'''

In [ ]:
'''
# 1등팀 feature selection by shap
X_train_select = pd.read_csv(os.path.abspath("../features")+'/X_train_shap_1st.csv', encoding='cp949')
X_test_select = pd.read_csv(os.path.abspath("../features")+'/X_test_shap_1st.csv', encoding='cp949')
'''

In [41]:
'''
# 2등팀 feature selection by shap
X_train_select = pd.read_csv(os.path.abspath("../features")+'/X_train_shap_2nd.csv', encoding='cp949')
X_test_select = pd.read_csv(os.path.abspath("../features")+'/X_test_shap_2nd.csv', encoding='cp949')
'''

In [80]:
'''
# 3등팀 feature selection by shap
X_train_select = pd.read_csv(os.path.abspath("../features")+'/X_train_shap_3rd.csv', encoding='cp949')
X_test_select = pd.read_csv(os.path.abspath("../features")+'/X_test_shap_3rd.csv', encoding='cp949')
'''

In [6]:
# merged feature selection by shap
X_train_select = pd.read_csv(os.path.abspath("../features")+'/X_train_shap_merge.csv', encoding='cp949')
X_test_select = pd.read_csv(os.path.abspath("../features")+'/X_test_shap_merge.csv', encoding='cp949')

### ensemble

In [7]:
y_train = y['age']

# 학습데이터 70%, 평가데이터 30%로 데이터 분할
X_train, X_valid, y_train, y_valid = train_test_split(X_train_select, y_train, test_size=0.3, random_state=0)

In [8]:
# 셀렉션 안한거
models = []

In [44]:
params = {
    'num_leaves': [7, 14, 21, 28, 31, 50],
    'learning_rate': [0.1, 0.03, 0.003],
    'max_depth': [-1, 3, 5],
    'n_estimators': [50, 100, 200, 500],
}

In [45]:
rand_search = RandomizedSearchCV(LGBMRegressor(), param_distributions=params, 
                                 scoring='neg_root_mean_squared_error', n_iter=5, random_state=1)
rand_search.fit(X_train, y_train)
print("Best estimator:\n{}".format(rand_search.best_estimator_))

Best estimator:
LGBMRegressor(max_depth=3, num_leaves=28)


### LGBM 

In [9]:
# LGBM
model =LGBMRegressor(max_depth=3, n_estimators=200, num_leaves=28,random_state=100)
model.fit(X_train, y_train, verbose=False)
print(np.sqrt(mean_squared_error(y_valid, model.predict(X_valid))))

models.append(model)

8.121734442621461


### CatBoost

In [10]:
# CatBoost
model = CatBoostRegressor(random_state=0)
model.fit(X_train, y_train, verbose=False)
print(np.sqrt(mean_squared_error(y_valid, model.predict(X_valid))))

models.append(model)

8.042343448708646


### Ridge

In [65]:
params = {'alpha': [0.05,0.1,1,5,8,10,12,15,20]}

In [66]:
rand_search = RandomizedSearchCV(Ridge(), param_distributions=params, 
                                 scoring='neg_root_mean_squared_error', n_iter=5, random_state=1)
rand_search.fit(X_train, y_train)
print("Best estimator:\n{}".format(rand_search.best_estimator_))

Best estimator:
Ridge(alpha=20)


In [11]:
model = Ridge(alpha=20,random_state=0)
model.fit(X_train, y_train)
print(np.sqrt(mean_squared_error(y_valid, model.predict(X_valid))))
models.append(model)

8.500892098933921


In [12]:
def experiment(i, h_size, num_hidden, activation, lr):
    model = tf.keras.Sequential()
    for _ in range(num_hidden):
        model.add(tf.keras.layers.Dense(h_size, activation=activation))
    model.add(tf.keras.layers.Dense(1, activation='linear'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='mse', metrics=[keras.metrics.RootMeanSquaredError()])
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),]
    hist = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), 
                 batch_size=128, epochs=300, callbacks=callbacks, shuffle=False, verbose=2)
    
    return min(hist.history['val_root_mean_squared_error'])

In [52]:
results = []
for i in range(3):
    h_sizes = [ 8, 16,32,64]
    h_size = np.random.choice(h_sizes)
    num_hidden = np.random.randint(0, 3)
    activation = np.random.choice(['relu'])
    lr = 10 ** np.random.randint(-5, 0)

    val_loss = experiment(i, h_size, num_hidden, activation, lr)
    results.append({
        'val_loss': val_loss,
        'h_size': h_size,
        'num_hidden': num_hidden,
        'activation': activation,
        'lr': lr
    })

Epoch 1/300
119/119 - 1s - loss: 42725.2539 - root_mean_squared_error: 206.7009 - val_loss: 416.3042 - val_root_mean_squared_error: 20.4035
Epoch 2/300
119/119 - 0s - loss: 426.6346 - root_mean_squared_error: 20.6551 - val_loss: 409.5042 - val_root_mean_squared_error: 20.2362
Epoch 3/300
119/119 - 0s - loss: 421.8242 - root_mean_squared_error: 20.5384 - val_loss: 404.9295 - val_root_mean_squared_error: 20.1229
Epoch 4/300
119/119 - 0s - loss: 416.7132 - root_mean_squared_error: 20.4136 - val_loss: 400.0155 - val_root_mean_squared_error: 20.0004
Epoch 5/300
119/119 - 0s - loss: 411.3111 - root_mean_squared_error: 20.2808 - val_loss: 394.8153 - val_root_mean_squared_error: 19.8700
Epoch 6/300
119/119 - 0s - loss: 405.6742 - root_mean_squared_error: 20.1414 - val_loss: 389.4109 - val_root_mean_squared_error: 19.7335
Epoch 7/300
119/119 - 0s - loss: 399.8659 - root_mean_squared_error: 19.9966 - val_loss: 383.8707 - val_root_mean_squared_error: 19.5926
Epoch 8/300
119/119 - 0s - loss: 393.9

Epoch 61/300
119/119 - 0s - loss: 86.9534 - root_mean_squared_error: 9.3249 - val_loss: 84.7697 - val_root_mean_squared_error: 9.2070
Epoch 62/300
119/119 - 0s - loss: 86.0457 - root_mean_squared_error: 9.2761 - val_loss: 83.8716 - val_root_mean_squared_error: 9.1581
Epoch 63/300
119/119 - 0s - loss: 85.1491 - root_mean_squared_error: 9.2276 - val_loss: 83.2394 - val_root_mean_squared_error: 9.1236
Epoch 64/300
119/119 - 0s - loss: 84.2889 - root_mean_squared_error: 9.1809 - val_loss: 82.9532 - val_root_mean_squared_error: 9.1079
Epoch 65/300
119/119 - 0s - loss: 83.5123 - root_mean_squared_error: 9.1385 - val_loss: 83.0476 - val_root_mean_squared_error: 9.1130
Epoch 66/300
119/119 - 0s - loss: 82.8459 - root_mean_squared_error: 9.1020 - val_loss: 83.4497 - val_root_mean_squared_error: 9.1351
Epoch 67/300
119/119 - 0s - loss: 82.2891 - root_mean_squared_error: 9.0713 - val_loss: 83.9864 - val_root_mean_squared_error: 9.1644
Epoch 68/300
119/119 - 0s - loss: 81.8261 - root_mean_squared_

Epoch 2/300
119/119 - 0s - loss: 541.6176 - root_mean_squared_error: 23.2727 - val_loss: 427.7203 - val_root_mean_squared_error: 20.6814
Epoch 3/300
119/119 - 0s - loss: 202.8277 - root_mean_squared_error: 14.2418 - val_loss: 160.3745 - val_root_mean_squared_error: 12.6639
Epoch 4/300
119/119 - 0s - loss: 176.4531 - root_mean_squared_error: 13.2836 - val_loss: 219.6918 - val_root_mean_squared_error: 14.8220
Epoch 5/300
119/119 - 0s - loss: 139.1396 - root_mean_squared_error: 11.7957 - val_loss: 796.7537 - val_root_mean_squared_error: 28.2268
Epoch 6/300
119/119 - 0s - loss: 163.2324 - root_mean_squared_error: 12.7762 - val_loss: 87.2617 - val_root_mean_squared_error: 9.3414
Epoch 7/300
119/119 - 0s - loss: 115.6737 - root_mean_squared_error: 10.7552 - val_loss: 89.4436 - val_root_mean_squared_error: 9.4575
Epoch 8/300
119/119 - 0s - loss: 111.9963 - root_mean_squared_error: 10.5828 - val_loss: 85.8809 - val_root_mean_squared_error: 9.2672
Epoch 9/300
119/119 - 0s - loss: 114.7483 - roo

In [53]:
results

[{'val_loss': 9.107863426208496,
  'h_size': 8,
  'num_hidden': 0,
  'activation': 'relu',
  'lr': 0.001},
 {'val_loss': 8.722222328186035,
  'h_size': 16,
  'num_hidden': 0,
  'activation': 'relu',
  'lr': 0.01},
 {'val_loss': 8.878175735473633,
  'h_size': 64,
  'num_hidden': 2,
  'activation': 'relu',
  'lr': 0.01}]

In [13]:
input = keras.Input(shape=(X_train.shape[1],))

x = keras.layers.Dense(16, activation='relu')(input) 
#함수모형의 API
output = keras.layers.Dense(1, activation='linear')(x) 
#회귀문제  항등함수 회귀는 아웃풋 뉴런이 하나여야함
model = keras.Model(input, output) 
#모델을 입력부터하지않고 X부터도 사용할 수 있음. 어디서부터 어디까지 모델로쓰겠다.

model.summary() 
#파라미터가 학습해야할 웨이트 308*16+1=4944 파라미터가 클수록 시간이오래걸린다.

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 981)]             0         
_________________________________________________________________
dense (Dense)                (None, 16)                15712     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 15,729
Trainable params: 15,729
Non-trainable params: 0
_________________________________________________________________


In [14]:
# loss, optimizer, metrics 설정 옵티마이저의 이름을쓰는건 
#옵티마이저의 디폴트값으로만 쓸것이고 
#함수로 쓸경우는 디폴트로 안하고 값을 바꾸겠다는것 .

#model.compile(loss='mse', optimizer='adam', metrics=[keras.metrics.RootMeanSquaredError()])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss='mse', metrics=[keras.metrics.RootMeanSquaredError()])

# batch size, epoch, 조기종료조건 등 설정
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),] 
#에퐄 돌렸음에도 불구하고 더이상 감소가없으면 스톱
#[keras.callbacks.ModelCheckpoint(filepath='best_nn_model.h5', monitor='val_loss', save_best_only=True)]

hist = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), 
                 batch_size=128, epochs=300, callbacks=callbacks, shuffle=False, verbose=2) #셔플 트루로하면 셔플하고함.
##배치사이즈를 크게하면 한번에 확돌아가니까 .. 배치사이즈를 작게하면 웨이트갱신은 빨리일어나지만 에포크가 느림.
##배치사이즈를 크게하면 크게할수록 좋긴한데 어느시점부터는 똑같다.

model.save('nn_model_rid.h5')

dnn_model = keras.models.load_model('nn_model_rid.h5')

print(np.sqrt(mean_squared_error(y_valid, dnn_model.predict(X_valid).flatten())))

Epoch 1/300
119/119 - 2s - loss: 1019.6760 - root_mean_squared_error: 31.9324 - val_loss: 257.8133 - val_root_mean_squared_error: 16.0566
Epoch 2/300
119/119 - 0s - loss: 146.9853 - root_mean_squared_error: 12.1238 - val_loss: 125.0434 - val_root_mean_squared_error: 11.1823
Epoch 3/300
119/119 - 0s - loss: 99.2573 - root_mean_squared_error: 9.9628 - val_loss: 82.9322 - val_root_mean_squared_error: 9.1067
Epoch 4/300
119/119 - 0s - loss: 84.4959 - root_mean_squared_error: 9.1922 - val_loss: 81.2028 - val_root_mean_squared_error: 9.0113
Epoch 5/300
119/119 - 0s - loss: 80.9139 - root_mean_squared_error: 8.9952 - val_loss: 77.4805 - val_root_mean_squared_error: 8.8023
Epoch 6/300
119/119 - 0s - loss: 80.0741 - root_mean_squared_error: 8.9484 - val_loss: 77.9432 - val_root_mean_squared_error: 8.8285
Epoch 7/300
119/119 - 0s - loss: 84.3058 - root_mean_squared_error: 9.1818 - val_loss: 80.0346 - val_root_mean_squared_error: 8.9462
Epoch 8/300
119/119 - 0s - loss: 76.4897 - root_mean_squared

In [15]:
models

[LGBMRegressor(max_depth=3, n_estimators=200, num_leaves=28, random_state=100),
 Ridge(alpha=20, random_state=0)]

### weight 찾기

In [16]:
# 최적의 가중치 찾기 
weights = []
rmse_best = 1000
for i in range(1, 10, 1):
    for j in range(1, 10, 1):
        for k in range(1, 10, 1):
            for l in range(1, 10, 1):
                if (i+j+k+l) != 10:
                    continue
                pred = (models[0].predict(X_valid) * i + models[1].predict(X_valid) * j + models[2].predict(X_valid) * k+dnn_model.predict(X_valid).flatten() * l)/10
                rmse = np.sqrt(mean_squared_error(y_valid, pred))
                print(rmse, i,j,k,l)            
                if rmse < rmse_best:
                    weights = [i,j,k,l]
                    rmse_best = rmse 
print(rmse_best, weights)

8.820086424079358 1 1 1 7
8.660172518294717 1 1 2 6
8.527561670621994 1 1 3 5
8.423543551429724 1 1 4 4
8.34918690765286 1 1 5 3
8.30528845876301 1 1 6 2
8.292331929250958 1 1 7 1
8.60485821309958 1 2 1 6
8.467757055123078 1 2 2 5
8.35932366369647 1 2 3 4
8.280684320812492 1 2 4 3
8.232692867502776 1 2 5 2
8.215886376412907 1 2 6 1
8.420461893802567 1 3 1 5
8.30771723929669 1 3 2 4
8.224853256140618 1 3 3 3
8.172778889708766 1 3 4 2
8.152084183658106 1 3 5 1
8.268960442755079 1 4 1 4
8.181953117148323 1 4 2 3
8.12582704538908 1 4 3 2
8.101224079535095 1 4 4 1
8.152188049808998 1 5 1 3
8.092062964729895 1 5 2 2
8.063550960854597 1 5 3 1
8.071652144437278 1 6 1 2
8.039250218262154 1 6 2 1
8.028443280451464 1 7 1 1
8.609857560633227 2 1 1 6
8.472866067850088 2 1 2 5
8.364528047431945 2 1 3 4
8.285967514019928 2 1 4 3
8.238036426878184 2 1 5 2
8.22127050759908 2 1 6 1
8.424375919645406 2 2 1 5
8.311713682588497 2 2 2 4
8.228919563276346 2 2 3 3
8.17690090262757 2 2 4 2
8.156246539789084 2 

In [20]:
# Ensemble: 가중평균 
weights = [0.2,0.6,0.1,0.1]
pred = models[0].predict(X_valid) * weights[0] + models[1].predict(X_valid) * weights[1] + models[2].predict(X_valid) * weights[2] + dnn_model.predict(X_valid).flatten() * weights[3] 
print(np.sqrt(mean_squared_error(y_valid, pred)))

7.872792048017793


### 전체 데이터로 재학습 (final)

In [21]:
y_train = y['age']

In [22]:
# 전체 학습데이터로 재학습
for i, m in enumerate(models):
    m.fit(X_train_select, y_train)
    models[i] = m

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Learning rate set to 0.067435
0:	learn: 10.2598393	total: 380ms	remaining: 6m 19s
1:	learn: 10.0991043	total: 764ms	remaining: 6m 21s
2:	learn: 9.9539631	total: 1.18s	remaining: 6m 31s
3:	learn: 9.8112647	total: 1.59s	remaining: 6m 36s
4:	learn: 9.6910135	total: 2.07s	remaining: 6m 51s
5:	learn: 9.5767935	total: 2.63s	remaining: 7m 15s
6:	learn: 9.4750804	total: 3.19s	remaining: 7m 32s
7:	learn: 9.3735968	total: 3.81s	remaining: 7m 52s
8:	learn: 9.2877464	total: 4.46s	remaining: 8m 10s
9:	learn: 9.1978463	total: 5.03s	remaining: 8m 17s
10:	learn: 9.1282181	total: 5.53s	remaining: 8m 17s
11:	learn: 9.0615924	total: 6.11s	remaining: 8m 23s
12:	learn: 9.0035142	total: 6.76s	remaining: 8m 33s
13:	learn: 8.9443504	total: 7.33s	remaining: 8m 36s
14:	learn: 8.8931415	total: 7.73s	remaining: 8m 27s
15:	learn: 8.8486558	total: 8.19s	remaining: 8m 23s
16:	learn: 8.8024656	total: 8.59s	remaining: 8m 16s
17:	learn: 8.7612933	total: 9.07s	remaining: 8m 14s
18:	learn: 8.7231669	total: 9.45s	remainin

157:	learn: 7.7133831	total: 1m 15s	remaining: 6m 44s
158:	learn: 7.7094083	total: 1m 16s	remaining: 6m 43s
159:	learn: 7.7055137	total: 1m 16s	remaining: 6m 41s
160:	learn: 7.7012192	total: 1m 16s	remaining: 6m 40s
161:	learn: 7.6963214	total: 1m 17s	remaining: 6m 39s
162:	learn: 7.6933276	total: 1m 17s	remaining: 6m 38s
163:	learn: 7.6901599	total: 1m 17s	remaining: 6m 37s
164:	learn: 7.6865541	total: 1m 18s	remaining: 6m 36s
165:	learn: 7.6822228	total: 1m 18s	remaining: 6m 35s
166:	learn: 7.6793972	total: 1m 18s	remaining: 6m 33s
167:	learn: 7.6758877	total: 1m 19s	remaining: 6m 33s
168:	learn: 7.6717790	total: 1m 19s	remaining: 6m 32s
169:	learn: 7.6683312	total: 1m 20s	remaining: 6m 31s
170:	learn: 7.6639369	total: 1m 20s	remaining: 6m 31s
171:	learn: 7.6606873	total: 1m 21s	remaining: 6m 30s
172:	learn: 7.6576642	total: 1m 21s	remaining: 6m 29s
173:	learn: 7.6550059	total: 1m 21s	remaining: 6m 28s
174:	learn: 7.6517972	total: 1m 22s	remaining: 6m 28s
175:	learn: 7.6475962	total:

311:	learn: 7.2005897	total: 2m 15s	remaining: 4m 59s
312:	learn: 7.1973376	total: 2m 16s	remaining: 4m 59s
313:	learn: 7.1950892	total: 2m 16s	remaining: 4m 58s
314:	learn: 7.1946469	total: 2m 16s	remaining: 4m 57s
315:	learn: 7.1919754	total: 2m 17s	remaining: 4m 57s
316:	learn: 7.1888704	total: 2m 17s	remaining: 4m 56s
317:	learn: 7.1862663	total: 2m 18s	remaining: 4m 55s
318:	learn: 7.1835799	total: 2m 18s	remaining: 4m 55s
319:	learn: 7.1812712	total: 2m 18s	remaining: 4m 55s
320:	learn: 7.1777470	total: 2m 19s	remaining: 4m 54s
321:	learn: 7.1731863	total: 2m 19s	remaining: 4m 53s
322:	learn: 7.1703291	total: 2m 19s	remaining: 4m 53s
323:	learn: 7.1669601	total: 2m 20s	remaining: 4m 52s
324:	learn: 7.1627497	total: 2m 20s	remaining: 4m 52s
325:	learn: 7.1597739	total: 2m 21s	remaining: 4m 51s
326:	learn: 7.1562639	total: 2m 21s	remaining: 4m 51s
327:	learn: 7.1540420	total: 2m 21s	remaining: 4m 50s
328:	learn: 7.1511498	total: 2m 22s	remaining: 4m 50s
329:	learn: 7.1483614	total:

464:	learn: 6.7756298	total: 3m 13s	remaining: 3m 42s
465:	learn: 6.7730110	total: 3m 13s	remaining: 3m 41s
466:	learn: 6.7703487	total: 3m 14s	remaining: 3m 41s
467:	learn: 6.7670765	total: 3m 14s	remaining: 3m 41s
468:	learn: 6.7648965	total: 3m 15s	remaining: 3m 40s
469:	learn: 6.7625945	total: 3m 15s	remaining: 3m 40s
470:	learn: 6.7589949	total: 3m 15s	remaining: 3m 39s
471:	learn: 6.7565980	total: 3m 16s	remaining: 3m 39s
472:	learn: 6.7539541	total: 3m 16s	remaining: 3m 38s
473:	learn: 6.7517990	total: 3m 16s	remaining: 3m 38s
474:	learn: 6.7499847	total: 3m 17s	remaining: 3m 38s
475:	learn: 6.7468226	total: 3m 17s	remaining: 3m 37s
476:	learn: 6.7445137	total: 3m 18s	remaining: 3m 37s
477:	learn: 6.7423956	total: 3m 18s	remaining: 3m 36s
478:	learn: 6.7395059	total: 3m 18s	remaining: 3m 36s
479:	learn: 6.7363160	total: 3m 19s	remaining: 3m 35s
480:	learn: 6.7336468	total: 3m 19s	remaining: 3m 35s
481:	learn: 6.7314159	total: 3m 20s	remaining: 3m 35s
482:	learn: 6.7288130	total:

617:	learn: 6.3958232	total: 4m 11s	remaining: 2m 35s
618:	learn: 6.3928695	total: 4m 11s	remaining: 2m 34s
619:	learn: 6.3911120	total: 4m 12s	remaining: 2m 34s
620:	learn: 6.3892466	total: 4m 12s	remaining: 2m 34s
621:	learn: 6.3866571	total: 4m 12s	remaining: 2m 33s
622:	learn: 6.3847773	total: 4m 13s	remaining: 2m 33s
623:	learn: 6.3825048	total: 4m 13s	remaining: 2m 32s
624:	learn: 6.3798756	total: 4m 13s	remaining: 2m 32s
625:	learn: 6.3772131	total: 4m 14s	remaining: 2m 31s
626:	learn: 6.3740149	total: 4m 14s	remaining: 2m 31s
627:	learn: 6.3722041	total: 4m 15s	remaining: 2m 31s
628:	learn: 6.3699762	total: 4m 15s	remaining: 2m 30s
629:	learn: 6.3678438	total: 4m 15s	remaining: 2m 30s
630:	learn: 6.3656865	total: 4m 16s	remaining: 2m 29s
631:	learn: 6.3636538	total: 4m 16s	remaining: 2m 29s
632:	learn: 6.3607022	total: 4m 16s	remaining: 2m 28s
633:	learn: 6.3582984	total: 4m 17s	remaining: 2m 28s
634:	learn: 6.3565632	total: 4m 17s	remaining: 2m 27s
635:	learn: 6.3542037	total:

770:	learn: 6.0486764	total: 5m 5s	remaining: 1m 30s
771:	learn: 6.0468572	total: 5m 5s	remaining: 1m 30s
772:	learn: 6.0452788	total: 5m 5s	remaining: 1m 29s
773:	learn: 6.0434157	total: 5m 6s	remaining: 1m 29s
774:	learn: 6.0408572	total: 5m 6s	remaining: 1m 29s
775:	learn: 6.0391934	total: 5m 6s	remaining: 1m 28s
776:	learn: 6.0370258	total: 5m 7s	remaining: 1m 28s
777:	learn: 6.0349454	total: 5m 7s	remaining: 1m 27s
778:	learn: 6.0329620	total: 5m 8s	remaining: 1m 27s
779:	learn: 6.0307795	total: 5m 8s	remaining: 1m 27s
780:	learn: 6.0280961	total: 5m 9s	remaining: 1m 26s
781:	learn: 6.0261310	total: 5m 9s	remaining: 1m 26s
782:	learn: 6.0240492	total: 5m 9s	remaining: 1m 25s
783:	learn: 6.0220842	total: 5m 9s	remaining: 1m 25s
784:	learn: 6.0193988	total: 5m 10s	remaining: 1m 24s
785:	learn: 6.0173652	total: 5m 10s	remaining: 1m 24s
786:	learn: 6.0154769	total: 5m 10s	remaining: 1m 24s
787:	learn: 6.0133980	total: 5m 11s	remaining: 1m 23s
788:	learn: 6.0112885	total: 5m 11s	remain

925:	learn: 5.7287237	total: 6m 1s	remaining: 28.9s
926:	learn: 5.7277231	total: 6m 2s	remaining: 28.5s
927:	learn: 5.7258607	total: 6m 2s	remaining: 28.1s
928:	learn: 5.7237969	total: 6m 2s	remaining: 27.7s
929:	learn: 5.7217069	total: 6m 3s	remaining: 27.3s
930:	learn: 5.7197822	total: 6m 3s	remaining: 27s
931:	learn: 5.7167259	total: 6m 4s	remaining: 26.6s
932:	learn: 5.7148958	total: 6m 4s	remaining: 26.2s
933:	learn: 5.7131405	total: 6m 4s	remaining: 25.8s
934:	learn: 5.7110762	total: 6m 5s	remaining: 25.4s
935:	learn: 5.7091065	total: 6m 5s	remaining: 25s
936:	learn: 5.7068293	total: 6m 5s	remaining: 24.6s
937:	learn: 5.7049185	total: 6m 6s	remaining: 24.2s
938:	learn: 5.7031109	total: 6m 6s	remaining: 23.8s
939:	learn: 5.7016785	total: 6m 6s	remaining: 23.4s
940:	learn: 5.6999794	total: 6m 7s	remaining: 23s
941:	learn: 5.6980301	total: 6m 7s	remaining: 22.6s
942:	learn: 5.6958635	total: 6m 7s	remaining: 22.2s
943:	learn: 5.6942666	total: 6m 7s	remaining: 21.8s
944:	learn: 5.6925

In [25]:
 #X_test_select = X_test_select.fillna(X_test_select.mean())

In [26]:
# 최종 예측모델
pred = models[0].predict(X_test_select) * weights[0] + models[1].predict(X_test_select) * weights[1] + models[2].predict(X_test_select) * weights[2] +dnn_model.predict(X_test_select).flatten() * weights[3] 

In [27]:
t = pd.Timestamp.now()
fname = f"submission_{t.month:02}{t.day:02}_{t.hour:02}{t.minute:02}.csv"
submissions = pd.concat([pd.Series(IDtest, name="custid"), pd.Series(pred, name="age")] ,axis=1)
submissions.to_csv(fname, index=False)
print("'{}' is ready to submit." .format(fname))

'submission_0615_2259.csv' is ready to submit.


- 0614 21:04 -> 우리 features, 최종스코어 8.69730
- 0614 20:02 -> 2등팀 features, 최종스코어 8.31725
- 0615 16:57 -> 1등팀 features, shap 최종스코어 8.26823


## END